In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

In [2]:
pd.set_option('display.max_columns', None)

In [6]:
df = pd.read_csv('DataSet_Hitelbiralat_joados.csv')

In [7]:
df

,id_client,id_shop,sex,marital_status,age,quant_dependants,education,flag_residencial_phone,area_code_residencial_phone,payment_day,shop_rank,residence_type,months_in_residence,flag_mothers_name,flag_fathers_name,flag_residence_town_working_town,flag_residence_state_working_state,months_in_the_job,profession_code,mate_income,flag_residencial_address_postal_address,flag_other_card,quant_banking_accounts,personal_reference_1,personal_reference_2,flag_mobile_phone,flag_contact_phone,personal_net_income,cod_application_booth,quant_additional_cards_in_the_application,flag_card_insurance_option,target
0,1,22,F,O,44,0,NaN,N,31,12,0,P,12,Y,Y,N,Y,48,731,0.0,Y,N,0,VERA,LUCIA,N,N,300.0,0,0,N,1
1,2,15,F,S,18,0,NaN,Y,31,20,0,P,216,Y,Y,Y,Y,12,853,0.0,Y,N,0,SARA,FELIPE,N,N,300.0,0,0,N,1
2,3,24,F,C,22,0,NaN,Y,31,8,0,P,48,Y,N,Y,Y,12,40,0.0,Y,N,0,HELENA,DOMINGOS SOGRA,N,N,229.0,0,0,N,1
3,4,12,F,C,47,0,NaN,N,31,25,0,P,180,Y,Y,N,Y,24,35,0.0,Y,N,0,JACI,VALERIA ALEXANDRA TRAJANO,N,N,304.0,0,0,N,1
4,5,16,F,S,28,0,NaN,Y,31,25,0,O,12,Y,Y,Y,Y,12,24,0.0,Y,N,0,MARCIA CRISTINA ZANELLA,SANDRO L P MARTINS,N,N,250.0,0,0,N,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49996,19,M,S,21,0,NaN,Y,31,12,0,P,120,Y,Y,Y,Y,12,218,0.0,Y,N,0,ALBA,DENILSON,N,N,234.0,0,0,N,1
49996,49997,22,F,C,40,0,NaN,Y,31,1,0,P,120,Y,Y,Y,Y,72,717,0.0,Y,N,0,JOANA,INES,N,N,1500.0,0,0,N,1
49997,49998,23,F,S,23,0,NaN,Y,31,28,0,P,264,Y,Y,Y,Y,12,991,0.0,Y,N,0,NOVINA,GLAUCIA,N,N,240.0,0,0,N,0
49998,49999,9,F,C,38,0,NaN,Y,5,28,0,P,48,Y,Y,Y,Y,204,40,0.0,Y,N,0,LILIANE FIGUEIREDO,MIRNA CARVALHO NASCIMENTO,N,N,616.0,0,0,N,1


In [8]:
df['target'].value_counts()

target
1    40126
0     9874
Name: count, dtype: int64

In [ ]:
# random baseline

In [9]:
np.random.seed(1)
df['pred_baseline_proba'] = np.random.rand(df.shape[0])

In [10]:
df['pred_baseline'] = df['pred_baseline_proba'].apply(lambda x: 1 if x >= 0.5 else 0)

In [11]:
accuracy_score(df['target'], df['pred_baseline'])

0.5034

In [12]:
recall_score(df['target'], df['pred_baseline'])

0.502865972187609

In [13]:
precision_score(df['target'], df['pred_baseline'])

0.8051875498802873

In [14]:
cm = confusion_matrix(df['target'], df['pred_baseline'])

In [15]:
(cm[0][0] + cm[1][1]) / cm.sum()

0.5034

In [16]:
cm

array([[ 4992,  4882],
       [19948, 20178]], dtype=int64)

In [17]:
(cm[0][0] + cm[1][1]) / cm.sum()

0.5034

In [18]:
cm[1][1] / cm[1].sum()

0.502865972187609

In [19]:
cm[1][1] / (cm[0][1] + cm[1][1])

0.8051875498802873

# adatelokeszites

In [20]:
dummy_cols_list = ['sex', 'marital_status', 'residence_type']
dummy_names = []

for col in dummy_cols_list:
    dummy_df = pd.get_dummies(df.loc[:, col], prefix=col, drop_first=True)
    dummy_names += dummy_df.columns.tolist()
    df = pd.concat([df, dummy_df], axis=1)

In [22]:
df

,id_client,id_shop,sex,marital_status,age,quant_dependants,education,flag_residencial_phone,area_code_residencial_phone,payment_day,shop_rank,residence_type,months_in_residence,flag_mothers_name,flag_fathers_name,flag_residence_town_working_town,flag_residence_state_working_state,months_in_the_job,profession_code,mate_income,flag_residencial_address_postal_address,flag_other_card,quant_banking_accounts,personal_reference_1,personal_reference_2,flag_mobile_phone,flag_contact_phone,personal_net_income,cod_application_booth,quant_additional_cards_in_the_application,flag_card_insurance_option,target,pred_baseline_proba,pred_baseline,sex_M,marital_status_D,marital_status_O,marital_status_S,marital_status_V,residence_type_C,residence_type_O,residence_type_P
0,1,22,F,O,44,0,NaN,N,31,12,0,P,12,Y,Y,N,Y,48,731,0.0,Y,N,0,VERA,LUCIA,N,N,300.0,0,0,N,1,0.417022,0,False,False,True,False,False,False,False,True
1,2,15,F,S,18,0,NaN,Y,31,20,0,P,216,Y,Y,Y,Y,12,853,0.0,Y,N,0,SARA,FELIPE,N,N,300.0,0,0,N,1,0.720324,1,False,False,False,True,False,False,False,True
2,3,24,F,C,22,0,NaN,Y,31,8,0,P,48,Y,N,Y,Y,12,40,0.0,Y,N,0,HELENA,DOMINGOS SOGRA,N,N,229.0,0,0,N,1,0.000114,0,False,False,False,False,False,False,False,True
3,4,12,F,C,47,0,NaN,N,31,25,0,P,180,Y,Y,N,Y,24,35,0.0,Y,N,0,JACI,VALERIA ALEXANDRA TRAJANO,N,N,304.0,0,0,N,1,0.302333,0,False,False,False,False,False,False,False,True
4,5,16,F,S,28,0,NaN,Y,31,25,0,O,12,Y,Y,Y,Y,12,24,0.0,Y,N,0,MARCIA CRISTINA ZANELLA,SANDRO L P MARTINS,N,N,250.0,0,0,N,1,0.146756,0,False,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49996,19,M,S,21,0,NaN,Y,31,12,0,P,120,Y,Y,Y,Y,12,218,0.0,Y,N,0,ALBA,DENILSON,N,N,234.0,0,0,N,1,0.402469,0,True,False,False,True,False,False,False,True
49996,49997,22,F,C,40,0,NaN,Y,31,1,0,P,120,Y,Y,Y,Y,72,717,0.0,Y,N,0,JOANA,INES,N,N,1500.0,0,0,N,1,0.497021,0,False,False,False,False,False,False,False,True
49997,49998,23,F,S,23,0,NaN,Y,31,28,0,P,264,Y,Y,Y,Y,12,991,0.0,Y,N,0,NOVINA,GLAUCIA,N,N,240.0,0,0,N,0,0.764897,1,False,False,False,True,False,False,False,True
49998,49999,9,F,C,38,0,NaN,Y,5,28,0,P,48,Y,Y,Y,Y,204,40,0.0,Y,N,0,LILIANE FIGUEIREDO,MIRNA CARVALHO NASCIMENTO,N,N,616.0,0,0,N,1,0.037197,0,False,False,False,False,False,False,False,True


In [21]:
X_cols = ['age', 'months_in_residence', 'months_in_the_job', 'personal_net_income', 'mate_income']
X_cols = X_cols + dummy_names
y_col = 'target'

In [ ]:
X = df.loc[:, X_cols]
y = df.loc[:, y_col]

In [ ]:
X.isnull().sum()

In [ ]:
model = LogisticRegression()

In [ ]:
pred = cross_val_predict(model, X, y, cv=4)

In [ ]:
pred

In [ ]:
df['pred_lr'] = pred

In [ ]:
def get_results(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    print('Acc:', round(acc, 4))
    print('Precision:', round(prec, 4))
    print('Recall:', round(rec, 4))
    print('CM:\n', cm)

In [ ]:
get_results(y, pred)

In [ ]:
pred_proba = cross_val_predict(model, X, y, cv=4, method='predict_proba')

In [ ]:
df['pred_lr_proba'] = pred_proba[:, 1]

In [ ]:
df['pred_lr_proba']

In [ ]:
df.groupby('target')['pred_lr_proba'].mean()

In [ ]:
plt.hist(df.loc[df['target']==1, 'pred_lr_proba'], bins=20)
plt.hist(df.loc[df['target']==0, 'pred_lr_proba'], bins=20);

In [ ]:
df['profit'] = df['target'].apply(lambda x: 15000 if x == 1 else -100000)

In [ ]:
df.sort_values('target', ascending=False)['profit'].cumsum().reset_index(drop=True).plot()
df.sort_values('pred_baseline_proba', ascending=False)['profit'].cumsum().reset_index(drop=True).plot()
df.sort_values('pred_lr_proba', ascending=False)['profit'].cumsum().reset_index(drop=True).plot()
plt.axhline(0, color='black')

In [ ]:
def create_profit_curve(df, sort_col):
    plot_df = df.sort_values(sort_col, ascending=False)['profit'].cumsum().reset_index(drop=True)
    plt.plot(plot_df, label=sort_col)
    plt.axhline(0, color='black')
    plt.legend()
    return None

In [ ]:
create_profit_curve(df, 'target')
create_profit_curve(df, 'pred_baseline_proba')
create_profit_curve(df, 'pred_lr_proba')

In [ ]:
dct = DecisionTreeClassifier(max_depth=5)

In [ ]:
pred = cross_val_predict(dct, X, y, cv=4)

In [ ]:
pred_proba = cross_val_predict(dct, X, y, cv=4, method='predict_proba')

In [ ]:
df['pred_dct_proba'] = pred_proba[:, 1]

In [ ]:
get_results(y, pred)

In [ ]:
create_profit_curve(df, 'target')
create_profit_curve(df, 'pred_baseline_proba')
create_profit_curve(df, 'pred_lr_proba')
create_profit_curve(df, 'pred_dct_proba')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
train_acc_list = []
test_acc_list = []
for i in range(1, 31):
    dct = DecisionTreeClassifier(max_depth=i)
    
    dct.fit(X_train, y_train)
    train_pred = dct.predict(X_train)
    test_pred = dct.predict(X_test)
    
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
    

In [ ]:
plt.plot(range(1, 31), train_acc_list, label='train')
plt.plot(range(1, 31), test_acc_list, label='test')
plt.legend()
plt.ylim(0, 1)

In [ ]:
dct = DecisionTreeClassifier(max_depth=2)

In [ ]:
dct.fit(X, y)

In [ ]:
plot_tree(dct, feature_names=X.columns, filled=True);

# KNN

In [ ]:
scaler = MinMaxScaler()

In [ ]:
model = KNeighborsClassifier(n_neighbors=3)

In [ ]:
pipeline = Pipeline([('scaler', scaler), ('model', model)])

In [ ]:
pred = cross_val_predict(pipeline, X, y, cv=4)
pred_proba = cross_val_predict(pipeline, X, y, cv=4, method='predict_proba')

In [ ]:
get_results(y, pred)

In [ ]:
df['pred_knn_proba'] = pred_proba[:, 1]

In [ ]:
create_profit_curve(df, 'target')
create_profit_curve(df, 'pred_baseline_proba')
create_profit_curve(df, 'pred_lr_proba')
create_profit_curve(df, 'pred_dct_proba')
create_profit_curve(df, 'pred_knn_proba')